<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/BiLSTM_WordLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

: 

MAKE DATASET

In [ ]:
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
import math
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt

dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((devData, np.concatenate((testData, trainData))))


# print(dataset)
gc.collect()      
max_num_words = 17
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/Copy of GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))

def checkDataPointExistence(patientID, split):
  for i in split:
    if(patientID == i[0]):
      return True
  return False

def getData(patientID, location):
  print("PatientID: " + str(int(patientID)))
  retData = [int(patientID)]
  textD = getTextData(patientID, location)
  textD = np.array(textD)
  audioD = getAudioData(patientID, location, textD)
  videoD = getVideoData(str(int(patientID)), location, textD)
  # patientD = np.concatenate((textD, audioD, videoD), axis = 1)
  # print("Final Patient Data: " + str(patientD.shape))
  # patientD = textD

  return textD,audioD,videoD

def getTextData(patientID, location):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
  file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))
  outputList = []

  for i in range(len(file)):
    sentence = file[i][3]
    words = str(sentence).split(" ")
    totalWordLength = 0
    for word in words: 
      totalWordLength += len(word)

    sentenceStart = file[i][0] 
    sentenceEnd = file[i][1]
    speaker = file[i][2]
    if speaker == 'Ellie':
      continue
    else:
      speaker = 1

    totalTime = sentenceEnd - sentenceStart
    timePerChar = totalTime/totalWordLength

    wordStart = sentenceStart
    for word in words:
      wordEnd = wordStart + (timePerChar * len(word))
      appender = [wordStart, wordEnd, speaker]
      vector = list(returnWordToVec(word))
      for v in vector:
        appender.append(v)
      # print(appender)
      # print(len(appender))
      outputList.append(appender)
      wordStart = wordEnd

  return outputList


def remove_StopWords(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence


def returnWordToVec(word):
  word = str(word)
  try:
    if word[0] == '<':
        word = word[1:]
    if word[-1] == '>':
        word = word[0:-1]
  except:
    dfdsd = 3
  if(word in model):
      return np.array(model[word])
  else:
    return np.zeros((300))

def audioDataHelper(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    return X
    
def getAudioData(patientID, location, textD):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_COVAREP.csv"
  data = pd.read_csv(fileName,header = None)
  data = data.iloc[:,:].values
  data = audioDataHelper(data)
  # print("Audio Raw Data:" + str(data.shape))
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.01)
    endIndex = math.ceil(sentenceEndTime/0.01)
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)
  
  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  # print("Audio Final Data:" + str(sentenceDatas.shape))

  return sentenceDatas

def getVideoDataHelper(patientID, location):
  root = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" 
  file1 = root + (patientID)+"_CLNF_AUs.txt"
  file2 = root + (patientID)+"_CLNF_features.txt"
  file3 = root + (patientID)+"_CLNF_features3D.txt"
  file4 = root + (patientID)+"_CLNF_gaze.txt"
  # file5 = root + (patientID)+"_CLNF_hog.bin"
  file6 = root + (patientID)+"_CLNF_pose.txt"
  data = processVideoData(file1)
  data = np.concatenate((data, processVideoData(file2)), 1)
  data = np.concatenate((data, processVideoData(file3)), 1)
  data = np.concatenate((data, processVideoData(file4)), 1)
  data = np.concatenate((data, processVideoData(file6)), 1)


  # try:
  #   hogFile = open(file5, 'r')
  # except:
  #   print("Hog tXT file")
  #   hogFile = open(root + (patientID)+"_CLNF_hog.txt", 'r')
  # hogData = np.fromfile(hogFile, dtype = np.uint32)
  # hogData = np.reshape(hogData, (-1, 4468))
  
  # data = np.concatenate((data, hogData), 1)
  return data

def processVideoData(filename):
  try:
    data = pd.read_csv(filename,delimiter=',', dtype=float)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
  except:
    print("Video Data corrupt, fixing.")
    data = pd.read_csv(filename,delimiter=',')
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))
            # print("se" , end = " ")    
  return X

def getVideoData(patientID, location, textD):
  data = getVideoDataHelper(patientID, location)
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.333)
    endIndex = math.ceil(sentenceEndTime/0.333)
    # print("Start Time: " + str(startIndex))
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    # This might be a possible error
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)
  
  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  # print("Video Final Data:" + str(sentenceDatas.shape))
  return sentenceDatas

# Xtrain = []
Ytrain = []
# Xtest = []
Ytest = []


audio_train = []
video_train = []
text_train = []

audio_test = []
video_test = []
text_test = []


lengths = []
for datapoint in dataset:
  # print(datapoint[0])
  if(checkDataPointExistence(datapoint[0], devData)):

    Data Point in Dev Set
    text,audio,video = getData(datapoint[0], dev_location)
    text = text[:,2:]
    audio_train.append(audio)
    video_train.append(video)
    text_train.append(text)
    Ytrain.append(datapoint[1])
    print(data)
  if(checkDataPointExistence(datapoint[0], testData)):
    # Data Point in Test Set
    text,audio,video = getData(datapoint[0], test_location)
    text = text[:,2:]
    audio_test.append(audio)
    video_test.append(video)
    text_test.append(text)
    Ytest.append(datapoint[1])
  elif(checkDataPointExistence(datapoint[0], trainData)):
    # Data Point in Train Set
    text,audio,video = getData(datapoint[0], train_location)
    text = text[:,2:]
    audio_train.append(audio)
    video_train.append(video)
    text_train.append(text)
    Ytrain.append(datapoint[1])
  # lengths.append(data.shape[0])
model = []
gc.collect()

def refactor(arr, size):
  temp = arr[0:min(len(arr),size), :]
  if (len(temp) < size):
    temp = np.concatenate((temp, np.zeros(((size - len(temp)), arr.shape[1]))), axis = 0 )
  return temp

numberOfSentences = 1700

for i in range(len(audio_train)):
  # Xtrain[i] = refactor(Xtrain[i], numberOfSentences)
  audio_train[i] = refactor(audio_train[i], numberOfSentences)
  video_train[i] = refactor(video_train[i], numberOfSentences)
  text_train[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtrain[i].shape)

for i in range(len(audio_test)):
  audio_test[i] = refactor(audio_train[i], numberOfSentences)
  video_test[i] = refactor(video_train[i], numberOfSentences)
  text_test[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtest[i].shape)

audio_test = np.array(audio_test)
video_test = np.array(video_test)
text_test = np.array(text_test)
# text_test = text_test[:,:,2:]

audio_train = np.array(audio_train)
video_train = np.array(video_train)
text_train = np.array(text_train)
# dataset = []
gc.collect()

print(audio_test.shape,video_test.shape,text_test.shape)
print(audio_train.shape,video_train.shape,text_train.shape)

dataset = []
devData = []
trainData = []
testdata = []
gc.collect()

Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)

gc.collect()

import sklearn
from sklearn import preprocessing

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train


audio_train = np.nan_to_num(audio_train)
video_train = np.nan_to_num(video_train)
text_train = np.nan_to_num(text_train)


audio_train, _ = upsample(audio_train,Ytrain)
video_train, _ = upsample(video_train,Ytrain)
text_train, Ytrain = upsample(text_train,Ytrain)

gc.collect()

for i in range(audio_train.shape[0]):
  audio_train[i] = sklearn.preprocessing.normalize(audio_train[i])
  video_train[i] = sklearn.preprocessing.normalize(video_train[i])
  text_train[i] = sklearn.preprocessing.normalize(text_train[i])



audio_test = np.nan_to_num(audio_test)
video_test = np.nan_to_num(video_test)
text_test = np.nan_to_num(text_test)


for i in range(audio_test.shape[0]):
  audio_test[i] = sklearn.preprocessing.normalize(audio_test[i])
  video_test[i] = sklearn.preprocessing.normalize(video_test[i])
  text_test[i] = sklearn.preprocessing.normalize(text_test[i])

gc.collect()

: 

BILSTM MODEL WITH GATING (WORD LEVEL)

In [ ]:
#Word Level

#Text+Audio+Video
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import layers

import numpy as np

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)
   
    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)  


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")
    
    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")


# Multiple Inputs
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

# first input model
input1 = Input(shape=(1700,74), name = 'Audio_input')
highway1 = Highway()(input1)
highway5 = Highway()(highway1)
highway6 = Highway()(highway5)
dense1 = Dense(74)(highway6)

# second input model
input2 = Input(shape=(1700,388), name = 'Video_input')
highway2 = Highway()(input2)
highway3 = Highway()(highway2)
highway4 = Highway()(highway3)
# dense8 = Dense(1000)(highway4)
# dense9 = Dense(500)(dense8)
dense7 = Dense(200)(highway4)
dense2 = Dense(74)(dense7) 

input3 = Input(shape = (1700,301), name = 'Text_input')
# dense4 = Dense(1000)(input3)
# dense5 = Dense(500)(dense4)
dense6 = Dense(150)(input3)
dense3 = Dense(74)(dense6)
# merge input models
merge = concatenate([dense1,dense2,dense3], axis = 1)
# interpretation model
lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)
bidire = Bidirectional(lstm)
output = Dense(1, activation='sigmoid')(bidire(merge))
model = Model(inputs=[input1, input2, input3], outputs=output)
# summarize layers
print(model.summary())
# plot graph
plot_model(model)
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')

# inp = np.array([audio_train,video_train,text_train], dtype = object)

filepath = '/content/drive/My Drive/MCA Dataset/checkFile.txt'
model.fit([audio_train,video_train,text_train],Ytrain, validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='min',
    baseline=None, restore_best_weights=True),epochs=50, batch_size = 2)




def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)


from sklearn.metrics import classification_report
pred = model.predict([audio_test,video_test,text_test])
pred2 = model.predict([audio_train,video_train,text_train])

print(classification_report(Ytest,Thresholding(pred,0.5)))
# print("TRAINING ACCCCCC        " , classification_report(Ytrain,Thresholding(pred2,0.5)))
# print(classification_report(Ytest,Thresholding(pred,0.6)))
# print(classification_report(Ytest,Thresholding(pred,0.4)))
# print(classification_report(Ytest,Thresholding(pred,0.3)))
# print(classification_report(Ytest,Thresholding(pred,0.7)))




: 